In [1]:
## Part I ###################################
## Data Processing ##########################

In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm # Processing

In [3]:
genome_scores_trans = pd.read_csv('Data/Movie_data/genome_scores_trans.csv',sep=',')
train_syn = pd.read_csv('Data/User_data/train_syn_5000.csv',sep=',')
genres = pd.read_csv('Data/Movie_data/genres.csv',sep=',')

print("genome_scores_trans.shape:",genome_scores_trans.shape)
print("train_syn.shape:",train_syn.shape)
print("genres.shape:",genres.shape)

genome_scores_trans.shape: (13816, 1129)
train_syn.shape: (617310, 5)
genres.shape: (58098, 22)


In [4]:
train_syn1 = train_syn.drop(['Unnamed: 0','timestamp'],axis=1)
train_syn1.head()

,userId,movieId,rating
0,24,31,4.0
1,24,50,4.5
2,24,318,4.5
3,24,527,4.5
4,24,858,4.0


In [5]:
genres1 = genres.drop(['Unnamed: 0'],axis=1)
genres1.head()

,movieId,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1,1,1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
movies = pd.read_csv('Data/ml-25m/movies.csv',sep=',')
print("movies.shape:",movies.shape)

movies.shape: (62423, 3)


In [8]:
# Based on train_syn1, filter genres1 with "movieId"

# Some movies in genres are not in train_syn, delect them first
movieId_genres = genres1['movieId'].unique()
train_syn2 = train_syn1[train_syn1['movieId'].isin(movieId_genres)]

movieId_train = train_syn2['movieId'].unique()
genres2 = genres1[genres1['movieId'].isin(movieId_train)]

movies1 = movies[movies['movieId'].isin(genres2['movieId'])]

print(genres2.shape)
print(movies1.shape)
print(len(movieId_train))

(13073, 21)
(13073, 3)
13073


In [ ]:
## Part II ##################################
## Traditional Algorithm ####################

In [ ]:
# Collaborative Filtering

In [9]:
def get_list_index_map(list):
    """
    将列表转为map
    :param list:输入列表
    :return:
    1. map item:index
    2. map_reverse index:item
    """
    map = {}
    map_reverse = {}
    for i in range(len(list)):
        map[list[i]] = i
        map_reverse[i] = list[i]
    return map, map_reverse

'''
def get_type_list():
    """
    获得所有类型的列表
    :return: 所有电影类型的list
    """
    type_list = []
    for item in genres_list:
        movie_types = item.split('|')
        for movie_type in movie_types:
            if movie_type not in type_list:
                type_list.append(movie_type)
    return type_list
'''

# 转为list
user_list = train_syn2['userId'].drop_duplicates().values.tolist()
movie_list = train_syn2['movieId'].drop_duplicates().values.tolist()
genres_list = movies1['genres'].values.tolist()

type_list = list(genres2.drop(['movieId'],axis=1).columns)

# 获得原index-数组index的map，便于后续处理
type_map, type_map_reverse = get_list_index_map(type_list)
user_map, user_map_reverse = get_list_index_map(user_list)
movie_map, movie_map_reverse = get_list_index_map(movie_list)

In [10]:
def get_rating_matrix():
    """
    构造评分矩阵
    :return: 二维数组，[i,j]表示user_i对movie_j的评分，缺省值为0
    """
    matrix = np.zeros((len(user_map.keys()), len(movie_map.keys())))
    for row in train_syn2.itertuples(index=True, name='Pandas'):
        user = user_map[getattr(row, "userId")] 
        movie = movie_map[getattr(row, "movieId")] # movieId 有问题？
        rate = getattr(row, "rating")
        matrix[user, movie] = rate
    print(matrix)
    return matrix

ratings_matrix = get_rating_matrix()

[[4.  4.5 4.5 ... 0.  0.  0. ]
 [0.  0.  0.  ... 0.  0.  0. ]
 [0.  0.  1.  ... 0.  0.  0. ]
 ...
 [0.  3.  5.  ... 0.  0.  0. ]
 [0.  0.  5.  ... 0.  0.  0. ]
 [0.  3.5 0.  ... 0.  0.  3.5]]


In [14]:
import math
from scipy import spatial
def get_user_sim_matrix(input_matrix):
    """
    构造用户相似度矩阵
    :param input_matrix: 输入矩阵，每i行代表用户i的特征向量
    :return: 对称矩阵，[i,j]=[j,i]=sim(user_i,user_j)
    """
    size = len(input_matrix)
    matrix = np.zeros((size, size))
    for i in tqdm(range(size)):
        for j in range(i + 1, size):
            sim = cosine_similarity(input_matrix[i], input_matrix[j])
            # sim = 1
            matrix[i, j] = sim
            matrix[j, i] = sim  # 对称矩阵，对角线为0
    return matrix


def cosine_similarity(list1, list2):
    """
    计算余弦相似度
    :param list1: 用户1的特征向量
    :param list2: 用户2的特征向量
    :return: 两个特征向量之间的余弦相似度
    """
    return 1 - spatial.distance.cosine(list1, list2)
#     res = 0
#     d1 = 0
#     d2 = 0
#     for index in range(len(list1)):
#         val1 = list1[index]
#         val2 = list2[index]
#         # for (val1, val2) in zip(list1, list2):
#         res += val1 * val2
#         d1 += val1 ** 2
#         d2 += val2 ** 2
#     return res / (math.sqrt(d1 * d2))

user_sim_matrix = get_user_sim_matrix(ratings_matrix)
np.savetxt('user_sim_matrix.csv', user_sim_matrix, delimiter = ',')

100%|██████████████████████████████████████████████████████████████████████████████| 4998/4998 [12:31<00:00,  6.65it/s]


In [15]:
def k_neighbor(matrix, index, k):
    """
    输入相似矩阵，读取k邻居的index
    :param matrix: 相似矩阵
    :param index: 目标index
    :param k:
    :return: list([k-index,相似度],....)
    """
    line = matrix[index]
    tmp = []
    for i in range(len(line)):
        tmp.append([i, line[i]])
    tmp.sort(key=lambda val: val[1], reverse=True)
    return tmp[:k]

In [16]:
def get_predict(matrix, index, k):
    """
    获取预测评分
    :param matrix: 相似矩阵
    :param index: 目标index
    :param k:k邻居的k
    :return: 根据KNN，获得对第index位用户评分的预测
    """
    neighbors = k_neighbor(matrix, index, k)
    all_sim = 0
    rate = [0 for i in range(len(ratings_matrix[0]))]
    for pair in neighbors:
        neighbor_index = pair[0] # 相似度高的用户
        neighbor_sim = pair[1]   # 对应用户的相似度
        all_sim += neighbor_sim  # 相似度累加
        rate += ratings_matrix[neighbor_index] * neighbor_sim # 相似度 * 评分 再累加
    rate /= all_sim # 最终评分=sum(相似度*评分)/sum(相似度)
    return rate

In [17]:
def get_CFRecommend(matrix, index, k, n):
    """
    获取推荐
    :param matrix: 相似矩阵
    :param index: 目标index
    :param k: k邻居的k
    :param n: 获取推荐的topN
    :return: list([movie_index,预测评分],...)
    """
    rate = get_predict(matrix, index, k)  # 获取预测评分
    for i in range(len(rate)):  # 如果用户已经评分过了，把预测评分设为0，也就是不会再推荐看过的电影
        if ratings_matrix[index][i] != 0:
            rate[i] = 0
    res = []
    for i in range(len(rate)):
        res.append([i, rate[i]])
    res.sort(key=lambda val: val[1], reverse=True)
    return res[:n]

In [ ]:
# Collaborative Filtering

In [18]:
def type_rank_map():
    """
    计算每类电影排名
    :return: map{'type':[(movie_id,平均评分,评分人数),...],...}
    """
    map = {}
    for t in type_list:
        map[t] = []
    for movie in range(len(genres_list)):        # 有多少部电影，genres_list就有多长（一一对应）
        print('正在处理电影', movie)
        # 计算该电影的用户均分
        rates = np.array(ratings_matrix)[:, movie]
        count = 0
        rate = 0
        for r in rates:
            if r != 0:
                rate += r
                count += 1
        if count != 0:  # 避免除0
            rate = rate / count               ####### 对每一部电影，计算平均分
        # 将(电影,评分,评分人数)加到对应的map中
        types = genres_list[movie].split('|')
        for t in types:
            map[t].append((movie, rate, count))
    # 排序，先根据评分高的优先，评分相同则评分人数多的优先
    for t in type_list:
        temp = map[t]
        temp.sort(key=lambda val: (val[1], val[2]), reverse=True)
        map[t] = temp
    return map    # map存有每一个type，每一个type键的值是(这个type的一部电影m，m的平均分，m的评分人数)

type_rank_map = type_rank_map()
import json
with open('type_rank_map.json','w') as file_obj:
    json.dump(type_rank_map,file_obj)

正在处理电影 0
正在处理电影 1
正在处理电影 2
正在处理电影 3
正在处理电影 4
正在处理电影 5
正在处理电影 6
正在处理电影 7
正在处理电影 8
正在处理电影 9
正在处理电影 10
正在处理电影 11
正在处理电影 12
正在处理电影 13
正在处理电影 14
正在处理电影 15
正在处理电影 16
正在处理电影 17
正在处理电影 18
正在处理电影 19
正在处理电影 20
正在处理电影 21
正在处理电影 22
正在处理电影 23
正在处理电影 24
正在处理电影 25
正在处理电影 26
正在处理电影 27
正在处理电影 28
正在处理电影 29
正在处理电影 30
正在处理电影 31
正在处理电影 32
正在处理电影 33
正在处理电影 34
正在处理电影 35
正在处理电影 36
正在处理电影 37
正在处理电影 38
正在处理电影 39
正在处理电影 40
正在处理电影 41
正在处理电影 42
正在处理电影 43
正在处理电影 44
正在处理电影 45
正在处理电影 46
正在处理电影 47
正在处理电影 48
正在处理电影 49
正在处理电影 50
正在处理电影 51
正在处理电影 52
正在处理电影 53
正在处理电影 54
正在处理电影 55
正在处理电影 56
正在处理电影 57
正在处理电影 58
正在处理电影 59
正在处理电影 60
正在处理电影 61
正在处理电影 62
正在处理电影 63
正在处理电影 64
正在处理电影 65
正在处理电影 66
正在处理电影 67
正在处理电影 68
正在处理电影 69
正在处理电影 70
正在处理电影 71
正在处理电影 72
正在处理电影 73
正在处理电影 74
正在处理电影 75
正在处理电影 76
正在处理电影 77
正在处理电影 78
正在处理电影 79
正在处理电影 80
正在处理电影 81
正在处理电影 82
正在处理电影 83
正在处理电影 84
正在处理电影 85
正在处理电影 86
正在处理电影 87
正在处理电影 88
正在处理电影 89
正在处理电影 90
正在处理电影 91
正在处理电影 92
正在处理电影 93
正在处理电影 94
正在处理电影 95
正在处理电影 96
正在处理电影 97
正在处理电影 98
正在处理电影 99
正在处理电影 100

正在处理电影 756
正在处理电影 757
正在处理电影 758
正在处理电影 759
正在处理电影 760
正在处理电影 761
正在处理电影 762
正在处理电影 763
正在处理电影 764
正在处理电影 765
正在处理电影 766
正在处理电影 767
正在处理电影 768
正在处理电影 769
正在处理电影 770
正在处理电影 771
正在处理电影 772
正在处理电影 773
正在处理电影 774
正在处理电影 775
正在处理电影 776
正在处理电影 777
正在处理电影 778
正在处理电影 779
正在处理电影 780
正在处理电影 781
正在处理电影 782
正在处理电影 783
正在处理电影 784
正在处理电影 785
正在处理电影 786
正在处理电影 787
正在处理电影 788
正在处理电影 789
正在处理电影 790
正在处理电影 791
正在处理电影 792
正在处理电影 793
正在处理电影 794
正在处理电影 795
正在处理电影 796
正在处理电影 797
正在处理电影 798
正在处理电影 799
正在处理电影 800
正在处理电影 801
正在处理电影 802
正在处理电影 803
正在处理电影 804
正在处理电影 805
正在处理电影 806
正在处理电影 807
正在处理电影 808
正在处理电影 809
正在处理电影 810
正在处理电影 811
正在处理电影 812
正在处理电影 813
正在处理电影 814
正在处理电影 815
正在处理电影 816
正在处理电影 817
正在处理电影 818
正在处理电影 819
正在处理电影 820
正在处理电影 821
正在处理电影 822
正在处理电影 823
正在处理电影 824
正在处理电影 825
正在处理电影 826
正在处理电影 827
正在处理电影 828
正在处理电影 829
正在处理电影 830
正在处理电影 831
正在处理电影 832
正在处理电影 833
正在处理电影 834
正在处理电影 835
正在处理电影 836
正在处理电影 837
正在处理电影 838
正在处理电影 839
正在处理电影 840
正在处理电影 841
正在处理电影 842
正在处理电影 843
正在处理电影 844
正在处理电影 845
正在处理电影 846

正在处理电影 1460
正在处理电影 1461
正在处理电影 1462
正在处理电影 1463
正在处理电影 1464
正在处理电影 1465
正在处理电影 1466
正在处理电影 1467
正在处理电影 1468
正在处理电影 1469
正在处理电影 1470
正在处理电影 1471
正在处理电影 1472
正在处理电影 1473
正在处理电影 1474
正在处理电影 1475
正在处理电影 1476
正在处理电影 1477
正在处理电影 1478
正在处理电影 1479
正在处理电影 1480
正在处理电影 1481
正在处理电影 1482
正在处理电影 1483
正在处理电影 1484
正在处理电影 1485
正在处理电影 1486
正在处理电影 1487
正在处理电影 1488
正在处理电影 1489
正在处理电影 1490
正在处理电影 1491
正在处理电影 1492
正在处理电影 1493
正在处理电影 1494
正在处理电影 1495
正在处理电影 1496
正在处理电影 1497
正在处理电影 1498
正在处理电影 1499
正在处理电影 1500
正在处理电影 1501
正在处理电影 1502
正在处理电影 1503
正在处理电影 1504
正在处理电影 1505
正在处理电影 1506
正在处理电影 1507
正在处理电影 1508
正在处理电影 1509
正在处理电影 1510
正在处理电影 1511
正在处理电影 1512
正在处理电影 1513
正在处理电影 1514
正在处理电影 1515
正在处理电影 1516
正在处理电影 1517
正在处理电影 1518
正在处理电影 1519
正在处理电影 1520
正在处理电影 1521
正在处理电影 1522
正在处理电影 1523
正在处理电影 1524
正在处理电影 1525
正在处理电影 1526
正在处理电影 1527
正在处理电影 1528
正在处理电影 1529
正在处理电影 1530
正在处理电影 1531
正在处理电影 1532
正在处理电影 1533
正在处理电影 1534
正在处理电影 1535
正在处理电影 1536
正在处理电影 1537
正在处理电影 1538
正在处理电影 1539
正在处理电影 1540
正在处理电影 1541
正在处理电影 1542
正在处理

正在处理电影 2144
正在处理电影 2145
正在处理电影 2146
正在处理电影 2147
正在处理电影 2148
正在处理电影 2149
正在处理电影 2150
正在处理电影 2151
正在处理电影 2152
正在处理电影 2153
正在处理电影 2154
正在处理电影 2155
正在处理电影 2156
正在处理电影 2157
正在处理电影 2158
正在处理电影 2159
正在处理电影 2160
正在处理电影 2161
正在处理电影 2162
正在处理电影 2163
正在处理电影 2164
正在处理电影 2165
正在处理电影 2166
正在处理电影 2167
正在处理电影 2168
正在处理电影 2169
正在处理电影 2170
正在处理电影 2171
正在处理电影 2172
正在处理电影 2173
正在处理电影 2174
正在处理电影 2175
正在处理电影 2176
正在处理电影 2177
正在处理电影 2178
正在处理电影 2179
正在处理电影 2180
正在处理电影 2181
正在处理电影 2182
正在处理电影 2183
正在处理电影 2184
正在处理电影 2185
正在处理电影 2186
正在处理电影 2187
正在处理电影 2188
正在处理电影 2189
正在处理电影 2190
正在处理电影 2191
正在处理电影 2192
正在处理电影 2193
正在处理电影 2194
正在处理电影 2195
正在处理电影 2196
正在处理电影 2197
正在处理电影 2198
正在处理电影 2199
正在处理电影 2200
正在处理电影 2201
正在处理电影 2202
正在处理电影 2203
正在处理电影 2204
正在处理电影 2205
正在处理电影 2206
正在处理电影 2207
正在处理电影 2208
正在处理电影 2209
正在处理电影 2210
正在处理电影 2211
正在处理电影 2212
正在处理电影 2213
正在处理电影 2214
正在处理电影 2215
正在处理电影 2216
正在处理电影 2217
正在处理电影 2218
正在处理电影 2219
正在处理电影 2220
正在处理电影 2221
正在处理电影 2222
正在处理电影 2223
正在处理电影 2224
正在处理电影 2225
正在处理电影 2226
正在处理

正在处理电影 2827
正在处理电影 2828
正在处理电影 2829
正在处理电影 2830
正在处理电影 2831
正在处理电影 2832
正在处理电影 2833
正在处理电影 2834
正在处理电影 2835
正在处理电影 2836
正在处理电影 2837
正在处理电影 2838
正在处理电影 2839
正在处理电影 2840
正在处理电影 2841
正在处理电影 2842
正在处理电影 2843
正在处理电影 2844
正在处理电影 2845
正在处理电影 2846
正在处理电影 2847
正在处理电影 2848
正在处理电影 2849
正在处理电影 2850
正在处理电影 2851
正在处理电影 2852
正在处理电影 2853
正在处理电影 2854
正在处理电影 2855
正在处理电影 2856
正在处理电影 2857
正在处理电影 2858
正在处理电影 2859
正在处理电影 2860
正在处理电影 2861
正在处理电影 2862
正在处理电影 2863
正在处理电影 2864
正在处理电影 2865
正在处理电影 2866
正在处理电影 2867
正在处理电影 2868
正在处理电影 2869
正在处理电影 2870
正在处理电影 2871
正在处理电影 2872
正在处理电影 2873
正在处理电影 2874
正在处理电影 2875
正在处理电影 2876
正在处理电影 2877
正在处理电影 2878
正在处理电影 2879
正在处理电影 2880
正在处理电影 2881
正在处理电影 2882
正在处理电影 2883
正在处理电影 2884
正在处理电影 2885
正在处理电影 2886
正在处理电影 2887
正在处理电影 2888
正在处理电影 2889
正在处理电影 2890
正在处理电影 2891
正在处理电影 2892
正在处理电影 2893
正在处理电影 2894
正在处理电影 2895
正在处理电影 2896
正在处理电影 2897
正在处理电影 2898
正在处理电影 2899
正在处理电影 2900
正在处理电影 2901
正在处理电影 2902
正在处理电影 2903
正在处理电影 2904
正在处理电影 2905
正在处理电影 2906
正在处理电影 2907
正在处理电影 2908
正在处理电影 2909
正在处理

...

正在处理电影 10321
正在处理电影 10322
正在处理电影 10323
正在处理电影 10324
正在处理电影 10325
正在处理电影 10326
正在处理电影 10327
正在处理电影 10328
正在处理电影 10329
正在处理电影 10330
正在处理电影 10331
正在处理电影 10332
正在处理电影 10333
正在处理电影 10334
正在处理电影 10335
正在处理电影 10336
正在处理电影 10337
正在处理电影 10338
正在处理电影 10339
正在处理电影 10340
正在处理电影 10341
正在处理电影 10342
正在处理电影 10343
正在处理电影 10344
正在处理电影 10345
正在处理电影 10346
正在处理电影 10347
正在处理电影 10348
正在处理电影 10349
正在处理电影 10350
正在处理电影 10351
正在处理电影 10352
正在处理电影 10353
正在处理电影 10354
正在处理电影 10355
正在处理电影 10356
正在处理电影 10357
正在处理电影 10358
正在处理电影 10359
正在处理电影 10360
正在处理电影 10361
正在处理电影 10362
正在处理电影 10363
正在处理电影 10364
正在处理电影 10365
正在处理电影 10366
正在处理电影 10367
正在处理电影 10368
正在处理电影 10369
正在处理电影 10370
正在处理电影 10371
正在处理电影 10372
正在处理电影 10373
正在处理电影 10374
正在处理电影 10375
正在处理电影 10376
正在处理电影 10377
正在处理电影 10378
正在处理电影 10379
正在处理电影 10380
正在处理电影 10381
正在处理电影 10382
正在处理电影 10383
正在处理电影 10384
正在处理电影 10385
正在处理电影 10386
正在处理电影 10387
正在处理电影 10388
正在处理电影 10389
正在处理电影 10390
正在处理电影 10391
正在处理电影 10392
正在处理电影 10393
正在处理电影 10394
正在处理电影 10395
正在处理电影 10396
正在处理电影 10397

正在处理电影 10953
正在处理电影 10954
正在处理电影 10955
正在处理电影 10956
正在处理电影 10957
正在处理电影 10958
正在处理电影 10959
正在处理电影 10960
正在处理电影 10961
正在处理电影 10962
正在处理电影 10963
正在处理电影 10964
正在处理电影 10965
正在处理电影 10966
正在处理电影 10967
正在处理电影 10968
正在处理电影 10969
正在处理电影 10970
正在处理电影 10971
正在处理电影 10972
正在处理电影 10973
正在处理电影 10974
正在处理电影 10975
正在处理电影 10976
正在处理电影 10977
正在处理电影 10978
正在处理电影 10979
正在处理电影 10980
正在处理电影 10981
正在处理电影 10982
正在处理电影 10983
正在处理电影 10984
正在处理电影 10985
正在处理电影 10986
正在处理电影 10987
正在处理电影 10988
正在处理电影 10989
正在处理电影 10990
正在处理电影 10991
正在处理电影 10992
正在处理电影 10993
正在处理电影 10994
正在处理电影 10995
正在处理电影 10996
正在处理电影 10997
正在处理电影 10998
正在处理电影 10999
正在处理电影 11000
正在处理电影 11001
正在处理电影 11002
正在处理电影 11003
正在处理电影 11004
正在处理电影 11005
正在处理电影 11006
正在处理电影 11007
正在处理电影 11008
正在处理电影 11009
正在处理电影 11010
正在处理电影 11011
正在处理电影 11012
正在处理电影 11013
正在处理电影 11014
正在处理电影 11015
正在处理电影 11016
正在处理电影 11017
正在处理电影 11018
正在处理电影 11019
正在处理电影 11020
正在处理电影 11021
正在处理电影 11022
正在处理电影 11023
正在处理电影 11024
正在处理电影 11025
正在处理电影 11026
正在处理电影 11027
正在处理电影 11028
正在处理电影 11029

正在处理电影 11584
正在处理电影 11585
正在处理电影 11586
正在处理电影 11587
正在处理电影 11588
正在处理电影 11589
正在处理电影 11590
正在处理电影 11591
正在处理电影 11592
正在处理电影 11593
正在处理电影 11594
正在处理电影 11595
正在处理电影 11596
正在处理电影 11597
正在处理电影 11598
正在处理电影 11599
正在处理电影 11600
正在处理电影 11601
正在处理电影 11602
正在处理电影 11603
正在处理电影 11604
正在处理电影 11605
正在处理电影 11606
正在处理电影 11607
正在处理电影 11608
正在处理电影 11609
正在处理电影 11610
正在处理电影 11611
正在处理电影 11612
正在处理电影 11613
正在处理电影 11614
正在处理电影 11615
正在处理电影 11616
正在处理电影 11617
正在处理电影 11618
正在处理电影 11619
正在处理电影 11620
正在处理电影 11621
正在处理电影 11622
正在处理电影 11623
正在处理电影 11624
正在处理电影 11625
正在处理电影 11626
正在处理电影 11627
正在处理电影 11628
正在处理电影 11629
正在处理电影 11630
正在处理电影 11631
正在处理电影 11632
正在处理电影 11633
正在处理电影 11634
正在处理电影 11635
正在处理电影 11636
正在处理电影 11637
正在处理电影 11638
正在处理电影 11639
正在处理电影 11640
正在处理电影 11641
正在处理电影 11642
正在处理电影 11643
正在处理电影 11644
正在处理电影 11645
正在处理电影 11646
正在处理电影 11647
正在处理电影 11648
正在处理电影 11649
正在处理电影 11650
正在处理电影 11651
正在处理电影 11652
正在处理电影 11653
正在处理电影 11654
正在处理电影 11655
正在处理电影 11656
正在处理电影 11657
正在处理电影 11658
正在处理电影 11659
正在处理电影 11660

正在处理电影 12215
正在处理电影 12216
正在处理电影 12217
正在处理电影 12218
正在处理电影 12219
正在处理电影 12220
正在处理电影 12221
正在处理电影 12222
正在处理电影 12223
正在处理电影 12224
正在处理电影 12225
正在处理电影 12226
正在处理电影 12227
正在处理电影 12228
正在处理电影 12229
正在处理电影 12230
正在处理电影 12231
正在处理电影 12232
正在处理电影 12233
正在处理电影 12234
正在处理电影 12235
正在处理电影 12236
正在处理电影 12237
正在处理电影 12238
正在处理电影 12239
正在处理电影 12240
正在处理电影 12241
正在处理电影 12242
正在处理电影 12243
正在处理电影 12244
正在处理电影 12245
正在处理电影 12246
正在处理电影 12247
正在处理电影 12248
正在处理电影 12249
正在处理电影 12250
正在处理电影 12251
正在处理电影 12252
正在处理电影 12253
正在处理电影 12254
正在处理电影 12255
正在处理电影 12256
正在处理电影 12257
正在处理电影 12258
正在处理电影 12259
正在处理电影 12260
正在处理电影 12261
正在处理电影 12262
正在处理电影 12263
正在处理电影 12264
正在处理电影 12265
正在处理电影 12266
正在处理电影 12267
正在处理电影 12268
正在处理电影 12269
正在处理电影 12270
正在处理电影 12271
正在处理电影 12272
正在处理电影 12273
正在处理电影 12274
正在处理电影 12275
正在处理电影 12276
正在处理电影 12277
正在处理电影 12278
正在处理电影 12279
正在处理电影 12280
正在处理电影 12281
正在处理电影 12282
正在处理电影 12283
正在处理电影 12284
正在处理电影 12285
正在处理电影 12286
正在处理电影 12287
正在处理电影 12288
正在处理电影 12289
正在处理电影 12290
正在处理电影 12291

正在处理电影 12847
正在处理电影 12848
正在处理电影 12849
正在处理电影 12850
正在处理电影 12851
正在处理电影 12852
正在处理电影 12853
正在处理电影 12854
正在处理电影 12855
正在处理电影 12856
正在处理电影 12857
正在处理电影 12858
正在处理电影 12859
正在处理电影 12860
正在处理电影 12861
正在处理电影 12862
正在处理电影 12863
正在处理电影 12864
正在处理电影 12865
正在处理电影 12866
正在处理电影 12867
正在处理电影 12868
正在处理电影 12869
正在处理电影 12870
正在处理电影 12871
正在处理电影 12872
正在处理电影 12873
正在处理电影 12874
正在处理电影 12875
正在处理电影 12876
正在处理电影 12877
正在处理电影 12878
正在处理电影 12879
正在处理电影 12880
正在处理电影 12881
正在处理电影 12882
正在处理电影 12883
正在处理电影 12884
正在处理电影 12885
正在处理电影 12886
正在处理电影 12887
正在处理电影 12888
正在处理电影 12889
正在处理电影 12890
正在处理电影 12891
正在处理电影 12892
正在处理电影 12893
正在处理电影 12894
正在处理电影 12895
正在处理电影 12896
正在处理电影 12897
正在处理电影 12898
正在处理电影 12899
正在处理电影 12900
正在处理电影 12901
正在处理电影 12902
正在处理电影 12903
正在处理电影 12904
正在处理电影 12905
正在处理电影 12906
正在处理电影 12907
正在处理电影 12908
正在处理电影 12909
正在处理电影 12910
正在处理电影 12911
正在处理电影 12912
正在处理电影 12913
正在处理电影 12914
正在处理电影 12915
正在处理电影 12916
正在处理电影 12917
正在处理电影 12918
正在处理电影 12919
正在处理电影 12920
正在处理电影 12921
正在处理电影 12922
正在处理电影 12923

In [19]:
type_rank_map = {}
with open('type_rank_map.json') as file_obj:
    type_rank_map = json.load(file_obj)

In [20]:
def get_user_favor_matrix():
    """
    构造用户偏好矩阵
    :return: [i,j]表示用户i对第j类型电影的喜爱程度
    """
    matrix = np.zeros((len(user_list), len(type_list)))
    for user in range((len(user_list))):
        weight = 0
        rating = ratings_matrix[user] # ratings_matrix存有所有user对所看的所有movie的rating，此处输出某个user对所看movie的评分
        for movie in range(len(rating)):
            if rating[movie] != 0:  # rating[movie]表示这个user对这个movie的rating
                # update favor
                types = genres_list[movie].split('|')    # 表示这个movie的所有type
                for t in types:
                    if t in type_map.keys():             # type_map当作字典理解，type_map[t]表示t的种类index i
                                                         # type_map: {type: i}
                        matrix[user][type_map[t]] += rating[movie]    # matrix的第user行，第type_map[t]列
                                                                      # 此处user和type_map[t]都是数字，表示matrix的位置
                                                                      # 将各个user对所有type的评分分别累加（对应movie才有分）
                        weight += rating[movie] # 单纯是累加所有评分
        # 完成以上循环后，得到了该user对所有type的rating总和
        matrix[user] /= weight # 该user行的各个types的评分，除以该user给出的所有types的评分总和，相当于评分比例
    return matrix   # 输出一个[user,type]的矩阵，内容是user对type的评分比例（以此代表偏好程度）

In [21]:
def get_CBRecommend(user_index, user_favor, type_rank, threshold=0):
    """
    获得基于内容的推荐，就推荐一个
    :param user_index: 目标用户
    :param user_favor: 用户偏好矩阵
    :param type_rank: 每类电影排名map
    :param threshold: 至少有threshold个人评分才算有效
    :return: list([movie_index,平均评分,评分人数],...)
    """
    favors = user_favor[user_index] # favors为特定user对一系列type的偏好程度（是个以type为index的list）
    max_val = 0 # 用max_val来选定最高的偏好程度值
    index = []  # 考虑如果有多个类型都一样喜欢，那么就挑可选出的评分最高的
    for i in range(len(favors)): # 遍历favors中的每一个type，选出最高偏好程度值对应的type的index（如偏好值相等可有多个index）
        if max_val != 0 and favors[i] == max_val:    
            index.append(i)
        elif favors[i] > max_val:
            max_val = favors[i]
            index = [i]
    candidate = []
    for i in index:
        tmp = type_rank[type_map_reverse[i]]  # 获取到该类排名list       type_map_reverse: {i: type}
                                              # type_rank输入type名会给出(movie, avg_rating, 评分人数)，按均分->人数排名
        for movie in tmp:
            if movie[2] > threshold and ratings_matrix[user_index][movie[0]] != 0: # movie[2]是这部电影的评分人数，大于定值
                                                                                   # user-movie matrix的取值不为0
                                                                                   #   表示该用户对这部电影评过分？？？？
                # 必须满足评分人数>threshold且用户没有看过
                candidate.append(movie)
                break
    # 排序，选择最优的
    candidate.sort(key=lambda val: (val[1], val[2]))     # 对满足上述条件筛选下来的所有电影，采用 高均分->多人数 进行排名
    return candidate[0]    # 排名完后，推荐排名最高者，通过更改0为0:n可以输出排名前n位

In [22]:
user_favor_matrix = get_user_favor_matrix()

In [23]:
# Output for CF & CBF

In [24]:
if __name__ == '__main__':
    output = [['userId', 'movieId']]
    user_sim_matrix_by_rating = np.loadtxt(open("./user_sim_matrix.csv", "rb"), delimiter=",")
    for user in user_list:
        res1 = get_CFRecommend(user_sim_matrix_by_rating, user_map[user], 10, 1)[0]
        res2 = get_CBRecommend(user_map[user], user_favor_matrix, type_rank_map)
        if res1[0] != res2[0]:
            output.append([user, movie_map_reverse[res1[0]]])
            output.append([user, movie_map_reverse[res2[0]]])
        else:
            output.append([user, movie_map_reverse[res1[0]]])
    np.savetxt('movie.csv', output, delimiter=',', fmt="%s")

In [25]:
output

[['userId', 'movieId'],
 [24, 1221],
 [24, 318],
 [36, 527],
 [36, 1198],
 [65, 79132],
 [65, 318],
 [76, 2858],
 [76, 32657],
 [192, 2959],
 [192, 720],
 [235, 2959],
 [235, 166643],
 [250, 457],
 [250, 296],
 [265, 589],
 [265, 912],
 [270, 2712],
 [270, 2028],
 [300, 33794],
 [300, 527],
 [319, 1288],
 [319, 1221],
 [346, 2959],
 [346, 858],
 [355, 7153],
 [355, 296],
 [404, 364],
 [404, 296],
 [428, 349],
 [428, 720],
 [443, 1136],
 [443, 1260],
 [480, 786],
 [480, 260],
 [522, 296],
 [522, 750],
 [608, 4993],
 [608, 135456],
 [672, 2571],
 [672, 750],
 [765, 110],
 [765, 1221],
 [767, 457],
 [767, 296],
 [777, 2571],
 [777, 750],
 [804, 7153],
 [804, 1198],
 [940, 434],
 [940, 296],
 [944, 6874],
 [944, 527],
 [953, 1079],
 [953, 750],
 [974, 1197],
 [974, 1932],
 [1056, 1291],
 [1056, 527],
 [1162, 2762],
 [1162, 527],
 [1174, 858],
 [1174, 750],
 [1218, 2959],
 [1218, 6016],
 [1344, 6539],
 [1344, 6016],
 [1374, 457],
 [1374, 527],
 [1426, 58559],
 [1426, 296],
 [1435, 3948],
 [